---
title: PARAFAC 2 on Wine
description: first proper attempt to decompose wine dataset.
project: parafac2
conclusion: ""
status: open
cdt: 2024-09-26T15:07:44
---

In [2]:
from database_etl import get_data
from pca_analysis.definitions import DB_PATH_UV
import polars as pl
import duckdb as db

con = db.connect(DB_PATH_UV, read_only=True)

ids = con.sql(
    """--sql
select inc_chm.runid from inc_chm join st using (samplecode) join ct using (vintage, wine) where varietal ='shiraz'
"""
).pl()["runid"]

shiraz_data = get_data(output="tuple", con=con)
shiraz_data[0][0].head()


id,mins,190,192,194,196,198,200,202,204,206,208,210,212,214,216,218,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,…,332,334,336,338,340,342,344,346,348,350,352,354,356,358,360,362,364,366,368,370,372,374,376,378,380,382,384,386,388,390,392,394,396,398,400,runid,path
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""c1e3411f-1d9e-4780-aace-a4650b…",0.0,-0.100836,-0.090286,-0.084877,-0.088319,-0.116237,-0.152491,-0.184111,-0.20989,-0.21103,-0.185363,-0.163682,-0.138558,-0.109747,-0.089183,-0.078201,-0.064656,-0.049725,-0.028498,-0.016235,-0.006683,0.004441,0.013769,0.020534,0.021987,0.022456,0.022985,0.020288,0.019573,0.018895,0.017829,0.018694,0.019729,0.020511,0.020102,0.018604,…,-0.014119,-0.012293,-0.010543,-0.010282,-0.012659,-0.012465,-0.012487,-0.01391,-0.015892,-0.014476,-0.010468,-0.010245,-0.015363,-0.020862,-0.02037,-0.018559,-0.021495,-0.018828,-0.016242,-0.017658,-0.016965,-0.014506,-0.011474,-0.00792,-0.009611,-0.013739,-0.01324,-0.013113,-0.006378,-0.005886,-0.009276,-0.004478,-0.005119,-0.014029,-0.0166,"""92""","""/Users/jonathan/mres_thesis/da…"
"""c1e3411f-1d9e-4780-aace-a4650b…",0.006667,-0.152573,-0.125825,-0.105329,-0.098512,-0.121959,-0.155978,-0.185095,-0.20992,-0.21144,-0.185713,-0.163361,-0.137843,-0.109114,-0.088133,-0.077836,-0.064626,-0.049308,-0.02791,-0.015162,-0.006095,0.004835,0.014164,0.020713,0.022106,0.022285,0.022762,0.020258,0.019759,0.019178,0.017688,0.018656,0.020258,0.021018,0.019997,0.018321,…,-0.014521,-0.012375,-0.011243,-0.010177,-0.011608,-0.012465,-0.012986,-0.014238,-0.01628,-0.014722,-0.010848,-0.009976,-0.014335,-0.020392,-0.019901,-0.017539,-0.020146,-0.017412,-0.015698,-0.018083,-0.017457,-0.014603,-0.01172,-0.008769,-0.010498,-0.0135,-0.011995,-0.011861,-0.005312,-0.005253,-0.008352,-0.004046,-0.005536,-0.013955,-0.01695,"""92""","""/Users/jonathan/mres_thesis/da…"
"""c1e3411f-1d9e-4780-aace-a4650b…",0.013333,-0.240289,-0.185803,-0.140652,-0.116594,-0.131525,-0.161052,-0.187069,-0.210479,-0.211842,-0.185877,-0.162601,-0.13683,-0.108719,-0.087172,-0.077404,-0.064924,-0.049241,-0.027366,-0.013918,-0.005186,0.005566,0.014625,0.020891,0.022441,0.022605,0.022814,0.02034,0.020161,0.019781,0.017896,0.018597,0.020579,0.021636,0.020109,0.018567,…,-0.014335,-0.011839,-0.01169,-0.010543,-0.011168,-0.012569,-0.013001,-0.013918,-0.016026,-0.0147,-0.010937,-0.009239,-0.013001,-0.01967,-0.018947,-0.016034,-0.01806,-0.015602,-0.015028,-0.018269,-0.017554,-0.014648,-0.012137,-0.009716,-0.011519,-0.013575,-0.011601,-0.011072,-0.004649,-0.004776,-0.007369,-0.004604,-0.006787,-0.012845,-0.015482,"""92""","""/Users/jonathan/mres_thesis/da…"
"""c1e3411f-1d9e-4780-aace-a4650b…",0.02,-0.356585,-0.264287,-0.187322,-0.140779,-0.143558,-0.16693,-0.189669,-0.211388,-0.212304,-0.185914,-0.161789,-0.136115,-0.108778,-0.086486,-0.076734,-0.065237,-0.049181,-0.026658,-0.012435,-0.003763,0.006653,0.015132,0.02107,0.023171,0.023767,0.023469,0.020839,0.020877,0.020735,0.018626,0.018701,0.020891,0.022501,0.020809,0.019446,…,-0.013709,-0.010818,-0.011899,-0.011131,-0.010848,-0.01239,-0.01239,-0.012964,-0.015043,-0.013836,-0.009969,-0.007741,-0.011496,-0.018552,-0.0173,-0.013843,-0.015303,-0.013098,-0.01362,-0.017412,-0.016406,-0.013933,-0.012144,-0.010222,-0.011474,-0.012778,-0.011019,-0.009969,-0.004143,-0.004366,-0.006728,-0.005744,-0.00757,-0.010036,-0.011787,"""92""","""/Users/jonathan/mres_thesis/da…"
"""c1e3411f-1d9e-4780-aace-a4650b…",0.026667,-0.481077,-0.34786,-0.237465,-0.167102,-0.156216,-0.172667,-0.192247,-0.21258,-0.212915,-0.186063,-0.161543,-0.135943,-0.108913,-0.086047,-0.076145,-0.065632,-0.049144,-0.025906,-0.010908,-0.002198,0.007786,0.015765,0.021353,0.024006,0.025213,0.024371,0.021406,0.02145,0.021674,0.019714,0